In [1]:
import os, sys
sys.path.append("/home/alexanderalbizu")
sys.path.append("/home/alexanderalbizu/.local/bin")
os.environ["WANDB_NOTEBOOK_NAME"] = "CNN_II.ipynb"
#!python setup.py develop 
#!pip install wandb
#!pip install 'monai[all]'
#!pip -q install vit_pytorch
#!pip -q install linformer
#!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install git+https://github.com/shijianjian/EfficientNet-PyTorch-3D

In [2]:
import json, glob, random, re, collections, time, cv2, torch

import numpy as np
import wandb as wb
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

from scipy.io import loadmat as lm
from enum import Enum
from torch import nn
from torch.utils import data as torch_data
from sklearn.model_selection import train_test_split
from torch.nn import functional as torch_functional

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from monai.networks import nets
from monai.data import (
    DataLoader,
    ImageDataset,
    CSVDataset,
)
from monai.transforms import (
    AddChannel,
    Compose,
    RandGaussianNoise,
    Resize,
    RemoveRepeatedChannel,
    Orientation,
    RandRotate90,
    RandBiasField,
    ScaleIntensity,
    ToDevice,
    EnsureType,
)
from efficientnet_pytorch_3d import EfficientNet3D
wb.login(); # 7e5f63e5846f29b034d98806712ab047df76834d

wandb: Currently logged in as: aalbizu. Use `wandb login --relogin` to force relogin


In [3]:
# Set data directory
rootDir = '/blue/camctrp/working/gullettj/ACT/derivatives/SVM/';
sdata = pd.read_csv(os.path.join(rootDir,'subjects_pre-mci_classification.csv')).to_numpy();
mri_types = ['T1', 'EPI']
T1_SIZE = (64,64,64)
T1_PATCH = (16,16,16)
EPI_SIZE = (51,51,1)
EPI_PAT = (17,17,17)
BATCH_SIZE = 4
N_EPOCHS = 5
SEED = 42
LEARNING_RATE = 5e-4
LR_DECAY = 0.9
samples_to_exclude = np.array([103744,106986,300142,101644,105903,106078,106817,101395,105554,204085])
exclude = np.equal(np.isin(sdata[:,0],samples_to_exclude),0);

num_of_cores = int(os.environ['SLURM_CPUS_PER_TASK'])

# Two binary labels for Healthy vs pre-MCI
subIdx = np.arange(sdata[exclude,0].shape[0]);
lab = np.array([sdata[:,1]]);
lab = lab[:,exclude]; 
labels = torch.nn.functional.one_hot(torch.as_tensor(lab.T)).float()

# Remove 20ch Head Coils
T1w_images = np.array([os.path.join(rootDir,'anat',''.join(['wc0csub-',str(sdata[s,0]),'_ses-01_T1w.nii'])) for s in range(sdata.shape[0])]);
T1w_images = T1w_images[exclude];

for s in range(sdata.shape[0]):
    EPI_images = np.array([os.path.join(rootDir,'func',''.join(['sub-',str(sdata[s,0]),'_ROIconnectivity.csv'])) ]);
EPI_images = EPI_images[exclude];

class Diagnosis(Enum):
    normal = 0
    preMCI = 1

# Plot Labels
plt.rcParams['figure.figsize'] = [20, 20]; plt.imshow(lab); 
plt.axis('off'); plt.title('Class Labels');

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 349

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(SEED)    

# Plot OG Images
fig, axes = plt.subplots(*[1,2], figsize=(5, 5), facecolor="white")

# T1w
axes[0].set_title(''.join(['sub-',str(sdata[0,0]),' T1 Data']));
nii = nib.load(T1w_images[0]); # Load Each Electrode
axes[0].imshow(nii.get_fdata()[:, :, 100].T, cmap="gray", origin="lower"); axes[0].axis('off'); 
# EPI
axes[1].set_title(''.join(['sub-',str(sdata[0,0]),' CONN Data']));
mat = pd.read_csv(EPI_images[0],delimiter=',',header=None); # Load Each Electrode
axes[1].imshow(mat, cmap="turbo"); axes[1].axis('off');
print('T1 Shape:',nii.shape,'\nCONN Shape:',mat.shape)

In [ ]:
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head *  heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        mlp_dim = 2048
        for _ in range(depth):
            #print (dim, mlp_dim)
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class Model(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = ((image_size // patch_size) ** 3); # 3D
        patch_dim = channels * patch_size ** 3

        self.patch_size = patch_size

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)
        #print (mlp_dim)
        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        #print (dim)
        self.to_cls_token = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, num_classes),
            nn.Dropout(dropout)
        )

    def forward(self, img, mask = None):
        p = self.patch_size # PAT_SIZE 1D
#         print (img.shape)
        x = rearrange(img, 'b c (h p1) (w p2) (d p3) -> b (h w d) (p1 p2 p3 c)', p1 = p, p2 = p, p3 = p) # SPLIT IMAGE INTO 3D PATCHES (b, # patchs, # patch features)
#         print (x.shape)
        x = self.patch_to_embedding(x) # Initialize Patch Weights
#         print (x.shape)
        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1) 
#         print (cls_tokens.shape)
        x = torch.cat((cls_tokens, x), dim=1)
#         print (x.shape)
#         print (self.pos_embedding.shape)
        x += self.pos_embedding # Gradient Step
        x = self.dropout(x)

        x = self.transformer(x)

        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer,
        criterion
    ):
        self.model = model
        self.device = device
        self.scaler = torch.cuda.amp.GradScaler()
        self.optimizer = optimizer
        self.lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=LR_DECAY)
        self.criterion = criterion

        self.best_valid_score = 0.0
        self.n_patience = 0
        self.lastmodel = None
        
        self.val_losses = []
        self.train_losses = []
        self.val_auc = []
        
        wb.init(project="ACT-CNN",
           config={
               "batch_size": BATCH_SIZE,
               "image_size": IM_SIZE,
               "learning_rate": LEARNING_RATE,
               "lr_decay": LR_DECAY,
               "dataset": "ACT",
           })
        wb.watch(self.model, log='all')
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):      
        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_time = self.train_epoch(train_loader)
            wb.log({'train_loss': train_loss, 'train_time': train_time})
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            wb.log({'valid_loss': valid_loss, 'valid_auc': valid_auc, 'valid_time': valid_time})
            
            self.train_losses.append(train_loss)
            self.val_losses.append(valid_loss)
            self.val_auc.append(valid_auc)
            
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s",
                n_epoch, train_loss, train_time
            )
            
            
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )

            if self.best_valid_score < valid_auc: 
                self.save_model(n_epoch, save_path, valid_loss, valid_auc)
                self.info_message(
                     "auc improved from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_score, valid_auc, self.lastmodel
                )
                self.best_valid_score = valid_auc
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break
        wb.run.log_code("."); wb.finish()
        
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0

        for step, batch in enumerate(train_loader, 1):
            X = batch[0].clone().float().to(self.device);
            targets = batch[1].to(self.device);
            self.optimizer.zero_grad()
            outputs = self.model(X).squeeze(1)
            loss = self.criterion(outputs, targets.float())            
            self.scaler.scale(loss).backward()
            
#             loss.backward()

            sum_loss += loss.detach().item()
            self.scaler.step(self.optimizer)
            self.scaler.update()
#             self.optimizer.step()
            
            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
            
        self.lr_scheduler.step()
        
        return sum_loss/len(train_loader), int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                targets = batch[1].to(self.device)

                output = self.model(batch[0].clone().detach().float().to(self.device)).squeeze(1)
                loss = self.criterion(output, targets.float())
                sum_loss += loss.detach().item()
                output = torch.sigmoid(output)
                
                y_all.extend(batch[1].tolist())
                outputs_all.extend(output.tolist())

            message = 'Valid Step {}/{}, valid_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
            
        auc = roc_auc_score(y_all, outputs_all)
        
        return sum_loss/len(valid_loader), auc, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path, loss, auc):
        self.lastmodel = f"{save_path}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
        
    def display_plots(self, mri_type):
        plt.figure(figsize=(10,5))
        plt.title("{}: Training and Validation Loss")
        plt.plot(self.val_losses,label="val")
        plt.plot(self.train_losses,label="train")
        plt.xlabel("iterations")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()
        plt.close()
        
        plt.figure(figsize=(10,5))
        plt.title("{}: Validation AUC-ROC")
        plt.plot(self.val_auc,label="val")
        plt.xlabel("iterations")
        plt.ylabel("AUC")
        plt.legend()
        plt.show()
        plt.close()
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

In [ ]:
nontest_list, test_list = train_test_split(subIdx, 
                                          test_size=0.05,
                                          stratify=lab.T,
                                          random_state=SEED)
nontest_label, test_label = train_test_split(lab.T, 
                                          test_size=0.05,
                                          stratify=lab.T,
                                          random_state=SEED)
train_list, valid_list = train_test_split(nontest_list, 
                                          test_size=0.10,
                                          stratify=nontest_label,
                                          random_state=SEED)
train_label, valid_label = train_test_split(nontest_label, 
                                          test_size=0.10,
                                          stratify=nontest_label,
                                          random_state=SEED);
print('train case split: ',sum(train_label)[0],':',len(train_label)-sum(train_label)[0])
print('valid case split: ',sum(valid_label)[0],':',len(valid_label)-sum(valid_label)[0])
print('test case split: ',sum(test_label)[0],':',len(test_label)-sum(test_label)[0],'\n')
del nontest_list, nontest_label; # Save RAM

In [ ]:
pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def build_model():
# #     model = nets.ViTAutoEnc(in_channels=1, img_size=IM_SIZE, patch_size=PAT_SIZE)
#     model = nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=1)
#     return model  
# class build_model():
#     def __init__(self):
#         super().__init__()
#         self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
#         n_features = self.net._fc.in_features
#         self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
#     def forward(self, x):
#         out = self.net(x)
#         return out

def train_mri_type(df_train, df_valid, train_lab, valid_lab, mri_type):
#     print(df_train.shape, df_valid.shape)
#     display(df_train.head())
#     display(df_valid.head())

    if mri_type == 'T1':
        # Define transforms
        train_transforms = Compose([
        ScaleIntensity(minv=0.0, maxv=1.0),
        AddChannel(),
        Resize(T1_SIZE),
        RandGaussianNoise(), 
        EnsureType(data_type='tensor')]);
        train_ds = ImageDataset(image_files=T1w_images[df_train], labels=train_lab[:,0], transform=train_transforms);
        model = Model(
        image_size = T1_SIZE[0],
        patch_size = T1_PATCH[0],
        num_classes = 1,
        dim = T1_SIZE[0]*4,
        depth = 2,
        heads = 16,
        mlp_dim = T1_SIZE[0]*8,
        channels = 1,
        dropout = 0.1,
        emb_dropout = 0.1
        )
    elif mri_type == 'EPI':
        # Define transforms
        train_transforms = Compose([EnsureType(data_type='tensor')]);
        train_ds = CSVDataset(src=list(EPI_images[df_train]), labels=train_lab[:,0], transform=train_transforms);
        model = Model(
        image_size = EPI_SIZE[0],
        patch_size = EPI_PAT[0],
        num_classes = 1,
        dim = EPI_SIZE[0]*4,
        depth = 2,
        heads = 16,
        mlp_dim = EPI_SIZE[0]*8,
        channels = 1,
        dropout = 0.1,
        emb_dropout = 0.1
        )
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_of_cores, pin_memory=pin_memory);

    # create a validation data loader
    if mri_type == 'T1':
        val_transforms = Compose([
        ScaleIntensity(minv=0.0, maxv=1.0),
        AddChannel(),
        Resize(T1_SIZE),
        EnsureType(data_type='tensor')]);
        valid_ds = ImageDataset(image_files=T1w_images[df_valid], labels=valid_lab[:,0], transform=val_transforms);
        model = Model(
        image_size = T1_SIZE[0],
        patch_size = T1_PATCH[0],
        num_classes = 1,
        dim = T1_SIZE[0]*4,
        depth = 2,
        heads = 16,
        mlp_dim = T1_SIZE[0]*8,
        channels = 1,
        dropout = 0.1,
        emb_dropout = 0.1
        )
    elif mri_type == 'EPI':
        val_transforms = Compose([EnsureType(data_type='tensor')]);
        valid_ds = CSVDataset(src=list(EPI_images[df_valid]), labels=valid_lab[:,0], transform=val_transforms);
        model = Model(
        image_size = EPI_SIZE[0],
        patch_size = EPI_PAT[0],
        num_classes = 1,
        dim = EPI_SIZE[0]*4,
        depth = 2,
        heads = 16,
        mlp_dim = EPI_SIZE[0]*8,
        channels = 1,
        dropout = 0.1,
        emb_dropout = 0.1
        )
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, num_workers=num_of_cores, pin_memory=pin_memory)

#     model = build_model()
   
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    criterion = torch_functional.binary_cross_entropy_with_logits

    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion
    )

    history = trainer.fit(
        N_EPOCHS, 
        train_loader, 
        valid_loader, 
        f"{mri_type}", 
        N_EPOCHS,
    )
    
    trainer.display_plots(mri_type)
    
    return trainer.lastmodel

modelfiles = None

if not modelfiles:
    modelfiles = [train_mri_type(train_list, valid_list, train_label, valid_label, m) for m in mri_types]
    print(modelfiles)

In [ ]:
def predict(modelfile, df, mri_type, split):
    print("Predict:", modelfile, mri_type, df.shape)
    
    # create a validation data loader
    if mri_type == 'T1':
        val_transforms = Compose([
        ScaleIntensity(minv=0.0, maxv=1.0),
        AddChannel(),
        Resize(T1_SIZE),
        EnsureType(data_type='tensor')]);
        test_ds = ImageDataset(image_files=T1w_images[test_list], labels=test_label[:,0], transform=val_transforms)
        model = Model(
        image_size = T1_SIZE[0],
        patch_size = T1_PATCH[0],
        num_classes = 1,
        dim = T1_SIZE[0]*4,
        depth = 2,
        heads = 16,
        mlp_dim = T1_SIZE[0]*8,
        channels = 1,
        dropout = 0.1,
        emb_dropout = 0.1
        )
    elif mri_type == 'EPI':
        val_transforms = Compose([EnsureType(data_type='tensor')]);
        test_ds = ImageDataset(src=list(EPI_images[test_list]), labels=test_label[:,0], transform=val_transforms)
        model = Model(
        image_size = EPI_SIZE[0],
        patch_size = EPI_PAT[0],
        num_classes = 1,
        dim = EPI_SIZE[0]*4,
        depth = 2,
        heads = 16,
        mlp_dim = EPI_SIZE[0]*8,
        channels = 1,
        dropout = 0.1,
        emb_dropout = 0.1
        )
    test_loader = DataLoader(test_ds, batch_size=test_batch, num_workers=num_of_cores, pin_memory=pin_memory)
   
    model.to(device)
    
    checkpoint = torch.load(modelfile)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    y_pred = []
    ids = []

    for e, batch in enumerate(data_loader,1):
        print(f"{e}/{len(data_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(torch.tensor(batch["X"]).float().to(device)).squeeze(1)).cpu().numpy().squeeze()
            if tmp_pred.size == 1:
                y_pred.append(tmp_pred)
            else:
                y_pred.extend(tmp_pred.tolist())
            ids.extend(batch["id"].numpy().tolist())
            
    preddf = pd.DataFrame({"sub": ids, "pred": y_pred}) 
    preddf = preddf.set_index("pred")
    return preddf

In [ ]:
df_pred = df_valid.set_index("sub")
df_pred["pred"] = 0
for m, mtype in zip(modelfiles,  mri_types):
    pred = predict(m, df_pred, mtype, "train")
    df_pred["pred"] += pred["pred"]
df_pred["pred"] /= len(modelfiles)
auc = roc_auc_score(df_pred["pred"], df_pred["pred"])
print(f"Validation ensemble AUC: {auc:.4f}")
print(df_pred["pred"])

In [ ]:
rearrange(img, 'b c (h p1) (w p2) (d p3) -> b (h w d) (p1 p2 p3 c)', p1 = PAT_SIZE[0], p2 = PAT_SIZE[0], p3 = PAT_SIZE[0])